<a href="https://colab.research.google.com/github/aniketAnvekar/Fake-News-Detection/blob/main/Fake_news_Detection_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
real_news = pd.read_csv('/content/drive/MyDrive/Fake news Dataset/True.csv')
fake_news = pd.read_csv('/content/drive/MyDrive/Fake news Dataset/Fake.csv')

In [ ]:
import re 
def removePrefix(text):
    pattern = r"^([A-Z]).*?-\s"
    text = re.sub(pattern, '', text)
    return text

real_news.text = real_news.text.apply(lambda x : removePrefix(x))
real_news.text[1]

'Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District Court in the meantime,” the official said, speaking on condition of anonymity. In Septem

In [ ]:
real_news['valid'] = 1
fake_news['valid'] = 0

news_source = pd.concat([real_news, fake_news], axis=0)
news_source = news_source[news_source[['text', 'title', 'date']].duplicated() == False]
news_source.text = news_source.text.apply(lambda x: np.nan if len(x.strip()) < 1 else x)
news_source = news_source.dropna()
news_source.drop(columns=['subject'], inplace=True)
news_source.duplicated().sum()

0

In [ ]:
news_source.to_csv('./Source.csv')

In [ ]:
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import string
import pandas as pd



In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
news_source = pd.read_csv('./Source.csv')
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)

In [ ]:
def token_freq(df, feature, valid):
    tar_texts = df[df.valid == valid][feature].values
    texts = ' '.join(tar_texts).lower()
    tokens = ''.join(char for char in texts if char not in punctuations).split()
    tokens_cleaned = [word for word in tokens if word not in stop_words]
    return pd.DataFrame(nltk.FreqDist(tokens_cleaned).most_common(10))[0]

real_title_freq = token_freq(news_source, 'title', 1)
real_text_freq = token_freq(news_source, 'text', 1)
fake_title_freq = token_freq(news_source, 'title', 0)
fake_text_freq = token_freq(news_source, 'text', 0)

In [ ]:
def count_freq_token(text, freq_df):
    text = text.lower()
    tokens = ''.join(char for char in text if char not in punctuations).split()
    count = 0
    for token in tokens:
        if token in freq_df.values:
            count += 1
    return count

In [ ]:
news_source['fake_title_token_freq'] = news_source.title.apply(lambda x : count_freq_token(x, fake_title_freq))
news_source['read_title_token_freq'] = news_source.title.apply(lambda x : count_freq_token(x, real_title_freq))
news_source['fake_text_token_freq'] = news_source.title.apply(lambda x : count_freq_token(x, fake_text_freq))
news_source['read_text_token_freq'] = news_source.title.apply(lambda x : count_freq_token(x, real_text_freq))

In [ ]:
def countPunctuation(text):
    ques = re.subn(r"\?", "", text)[1]
    exclam = re.subn(r"\!", "", text)[1]
    return ques, exclam

In [ ]:
news_source[['title_ques_num', 'title_exclam_num']] = news_source.title.apply(lambda x : pd.Series(countPunctuation(x)))
news_source[['text_ques_num', 'text_exclam_num']] = news_source.text.apply(lambda x : pd.Series(countPunctuation(x)))

In [ ]:
news_source['title_len'] = news_source.title.apply(lambda x : len(x))
news_source['title_ratio'] = news_source.text.apply(lambda x : len(x))
news_source['title_ratio'] = news_source['title_len'] / news_source['title_ratio']

In [ ]:
news_source.to_csv('./NewsAna.csv')

In [ ]:
news_source.corr().valid

Unnamed: 0               0.120031
valid                    1.000000
fake_title_token_freq   -0.473264
read_title_token_freq    0.194594
fake_text_token_freq    -0.061342
read_text_token_freq     0.064952
title_ques_num          -0.144762
title_exclam_num        -0.238136
text_ques_num           -0.324997
text_exclam_num         -0.237945
title_len               -0.595802
title_ratio             -0.066529
Name: valid, dtype: float64

In [ ]:
news_source.drop(columns=['Unnamed: 0', 'title', 'text', 'date'], inplace=True)

In [ ]:
news_source.reset_index()
news_source = news_source.sample(frac=1.)

In [ ]:
pip install transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 636 kB 53.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler
import torch

batch_size = 64
epoch_num = 4
max_seq_length = 128

news_source = pd.read_csv('./NewsAna.csv')
news_source.reset_index()
news_source = news_source.sample(frac=1.)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model_path = "bert-base-uncased"

In [ ]:
class NewsModel(torch.nn.Module):
    def __init__(self, bert_model=bert_model_path, num_class=1):
        super(NewsModel, self).__init__()
        # Load pre-trained model (from huggingface)
        self.bert_layer = AutoModel.from_pretrained(pretrained_model_name_or_path=bert_model)
        # Maybe you can add a little feature data to get the final result
        self.bert_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=bert_model)
        self.mid_dim = self.bert_config.hidden_size
        # Perform final classification
        self.output = torch.nn.Sequential(
            torch.nn.Linear(self.mid_dim, self.mid_dim//2),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(self.mid_dim//2, self.mid_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(self.mid_dim, num_class),
            torch.nn.Sigmoid()
        )
    def forward(self, input_ids, attn_mask=None):
        bert_out = self.bert_layer(input_ids=input_ids, attention_mask=attn_mask)[1]
        output = self.output(bert_out)
        return output

In [ ]:
def covertTokenFormat(df, bert_model_path, max_seq_len):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=bert_model_path)
    texts = df.text.tolist()
    text_tokens = tokenizer(texts, padding='max_length', max_length=max_seq_len, truncation=True, return_tensors="pt")
    labels = torch.tensor(df.valid.values, dtype=torch.float)
    return text_tokens, labels

In [ ]:
def news_bert_report(pred, label):
    tar_y = label.squeeze()
    pred_y = []
    for item in pred.squeeze():
        if item >= 0.5:
            pred_y.append(1)
        else:
            pred_y.append(0)
    f1 = f1_score(tar_y, pred_y)
    print("f1-score: ", f1)
    acc = accuracy_score(tar_y, pred_y)
    print("accuracy: ", acc)
    return f1, acc

In [ ]:
# create model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

news_model = NewsModel(bert_model=bert_model_path).to(device)
optimiser = AdamW(news_model.parameters(), lr=1e-5)
# parepare the train, test set
train_set, test_set = train_test_split(news_source, test_size=0.2, random_state=7)
# dataloader - training set
text_tokens, labels = covertTokenFormat(train_set, bert_model_path, max_seq_length)
train_data = TensorDataset(text_tokens.input_ids, text_tokens.attention_mask, labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from torch.nn import functional as F

def train_bert(bert, optimiser, train_dataloader, save_check_point=True):
    bert = bert.to(device)
    bert.train()
    for epoch in range(epoch_num): 
        epoch_loss = 0
        pred_lis = torch.Tensor()
        label_lis = torch.Tensor()
        for batch, (token_ids, attn_mask, label) in enumerate(train_dataloader):
            # keep all the parameters in the same device
            token_ids = token_ids.to(device)
            attn_mask = attn_mask.to(device)
            label = label.to(device)
            # the output will be in the same device with the model
            outputs = bert(token_ids, attn_mask)
            loss = F.binary_cross_entropy(outputs.squeeze(), label)
            # do the backprop and update the parameters
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            epoch_loss += loss.cpu().data.numpy()
            pred_lis = torch.cat([pred_lis, outputs.cpu().squeeze()])
            label_lis = torch.cat([label_lis, label.cpu().squeeze()])
            if batch % 50 == 0:
                print("Current batch loss :", loss.cpu().data.numpy())
        print("Now epoch :", epoch+1, " Total epoch loss is: ", epoch_loss)
        news_bert_report(pred_lis.detach().numpy(), label_lis.detach().numpy())
        if save_check_point:
            torch.save({'model_state_dict': bert.state_dict()}, './bert_model_' + str(epoch) + '.weights')
    return bert

In [ ]:
bert.summary()

NameError: ignored

In [ ]:
news_model = train_bert(
    news_model,
    optimiser,
    train_dataloader
)

Current batch loss : 0.69451
Current batch loss : 0.40746462
Current batch loss : 0.18787476
Current batch loss : 0.123503625
Current batch loss : 0.031932704
Current batch loss : 0.022840716
Current batch loss : 0.014713131
Current batch loss : 0.009728976
Current batch loss : 0.040500127
Current batch loss : 0.012250946
Now epoch : 1  Total epoch loss is:  62.845495854038745
f1-score:  0.9744543870529518
accuracy:  0.9720327201008762
Current batch loss : 0.0057031577
Current batch loss : 0.009065968
Current batch loss : 0.0041634487
Current batch loss : 0.0030161282
Current batch loss : 0.019048307
Current batch loss : 0.0022180916
Current batch loss : 0.0057903663
Current batch loss : 0.0058766445
Current batch loss : 0.00202137
Current batch loss : 0.0021976647
Now epoch : 2  Total epoch loss is:  6.994901177473366
f1-score:  0.9964801126363956
accuracy:  0.9961201461411621
Current batch loss : 0.0070582395
Current batch loss : 0.0025160904
Current batch loss : 0.001536726
Current 

In [ ]:
test_tokens, test_labels = covertTokenFormat(test_set, bert_model_path, max_seq_length)
test_data = TensorDataset(test_tokens.input_ids, test_tokens.attention_mask, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

# close the gradient
with torch.no_grad():
    news_model.eval()
    pred_lis = torch.Tensor()
    label_lis = torch.Tensor()
    for batch, (token_ids, attn_mask, label) in enumerate(test_dataloader):
        token_ids = token_ids.to(device)
        attn_mask = attn_mask.to(device)
        label = label.to(device)
        outputs = news_model(token_ids, attn_mask)
        pred_lis = torch.cat([pred_lis, outputs.cpu().squeeze()])
        label_lis = torch.cat([label_lis, label.cpu().squeeze()])

    news_bert_report(pred_lis.detach().numpy(), label_lis.detach().numpy())

f1-score:  0.9950845222395396
accuracy:  0.9946980473296263


In [ ]:
pred_lis

tensor([2.4227e-04, 2.3970e-04, 2.9743e-04,  ..., 9.9944e-01, 9.9955e-01,
        9.9955e-01])

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(pred_lis,label_lis)
print(cm)
sns.heatmap(cm_tfidf, annot=True, cmap="YlGnBu")

ValueError: ignored